In [1]:
# Read the CSV file from the Robert Wood Johnson Foundation program.

import pandas as pd

url="https://www.countyhealthrankings.org/sites/default/files/analytic_data2019.csv"
dfLE = pd.read_csv(url, low_memory=False)

In [2]:
# Set the column names equal to the data in the first row. Then remove the first row.

dfLE.columns = dfLE.iloc[0]
dfLE.drop(dfLE.index[0], inplace=True)

In [3]:
# Create a dataframe specific to North Carolina (which is state code 37)

NorthCarolina = (dfLE.loc[dfLE['statecode']=='37'])[['countycode', 'county', 'v147_rawvalue']]
NorthCarolina.reset_index(inplace=True)
NorthCarolina.drop(columns=['index'], inplace=True)

# Drop the first row as this does not contain county data.

NorthCarolina.drop(NorthCarolina.index[0], inplace=True)

In [4]:
# Are there any NaN values ?
print(NorthCarolina.loc[NorthCarolina['v147_rawvalue'].isna()])

Empty DataFrame
Columns: [countycode, county, v147_rawvalue]
Index: []


In [5]:
# Convert the county code to an integer.
# Convert the life expectancy to a float.

NorthCarolina.rename(columns = {'v147_rawvalue':'Life Expectancy'}, inplace=True)
NorthCarolina['countycode'] = NorthCarolina['countycode'].astype(int)
NorthCarolina['Life Expectancy'] = NorthCarolina['Life Expectancy'].astype(float)

In [6]:
# Create a list of dictionaries containing the data for each county.

NCData = NorthCarolina.to_dict('records')

In [7]:
# Read the geographical information.

import geopandas as gp

url = "https://raw.github.com/JerryGreenough/Mecklenburg-County/master/us500k.json"
us = gp.read_file(url)

In [8]:
# Select the North Carolina counties. Convert the county FIPS number to an integer.

NorthCarolinaCounties = us.loc[us['STATE']=='37']
NorthCarolinaCounties['COUNTY'] = NorthCarolinaCounties['COUNTY'].astype(int)

C:\Users\jgree\Anaconda3\Lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [9]:
# Create a dictionary of polygons for North Carolina.

polygons = dict()

for i in range(len(NorthCarolinaCounties)):
    if type(NorthCarolinaCounties.iloc[i,6]).__name__ == 'Polygon':
        
        aaa = list(NorthCarolinaCounties.iloc[i,6].exterior.coords)
        polygons[NorthCarolinaCounties.iloc[i,2]] = aaa
    else:
        aaa = list(NorthCarolinaCounties.iloc[i,6][0].exterior.coords)
        polygons[NorthCarolinaCounties.iloc[i,2]] = aaa

In [10]:
# For each county listed in NCData.

for xxx in NCData:
    xxx['polygon'] = polygons[xxx['countycode']]

In [11]:
# Dependencies
# ----------------------------------
# Imports the method used for connecting to DBs
from sqlalchemy import create_engine

# Imports the methods needed to abstract classes into tables
from sqlalchemy.ext.declarative import declarative_base

# Allow us to declare column types
from sqlalchemy import Column, Integer, String, Float, ARRAY

In [12]:
Base = declarative_base()

# Working with the SQLAlchemy ORM.
# Define a mapped class, which maps to rows of the NC Counties database table.

# Creates an NC county class which will serve as the anchor points for our Tables.
class NCCounty(Base):
    __tablename__ = 'NC Counties'
    id = Column(Integer, primary_key=True)
    name = Column(String(255))
    boundary = Column(ARRAY(Float))
    
    
class NCLifeExpectancy(Base):
    __tablename__ = 'NC Life Expectancy'
    id = Column(Integer, primary_key=True)
    life_expectancy = Column(Float)

In [13]:
from dbconfig import sqlConnection
engine = create_engine(sqlConnection)
conn = engine.connect()

In [14]:
Base.metadata.drop_all(engine)
Base.metadata.create_all(engine)

In [15]:
# Create a Session Object to Connect to DB
# ----------------------------------
from sqlalchemy.orm import Session
session = Session(bind=engine)

In [16]:
for xx in NCData:
    ncc = NCCounty(id=int(xx['countycode']), name=xx['county'], boundary=xx['polygon'])
    session.add(ncc)
    
session.commit()

In [17]:
for xx in NCData:
    ledat = NCLifeExpectancy(id=int(xx['countycode']), life_expectancy=xx['Life Expectancy'])
    session.add(ledat)
    
session.commit()